# Snapshot Sync Demonstration

This notebook demonstrates the `snapshot_sync` method of `AdapterStore`, which synchronizes the table to match a complete snapshot of data.

## Key Characteristics of snapshot_sync

- **Full Synchronization**: Updates, inserts, AND deletes to match the snapshot exactly
- **Soft Deletes**: Records missing from snapshot are marked with `content=None` (not physically removed)
- **No Timestamp Checking**: Unlike `incremental_update`, snapshot_sync does NOT require or check timestamps for gating updates
- **Timestamp Writing**: Does write `last_modified` timestamps to all changed records (auto-generated if not provided)
- **Idempotent**: Applying the same snapshot twice produces no changes
- **Namespace-Scoped**: Only affects records in the specified namespace

**Use case**: Full harvests where you receive a complete dataset and want the table to match exactly (e.g., nightly full dumps).

In [ ]:
import sys
from uuid import uuid1
import pyarrow as pa
from adapters.utils.adapter_store import AdapterStore
from adapters.utils.iceberg import get_local_table

# Create a local Iceberg table for testing
table_name = f"demo_{str(uuid1())[:8]}"
table = get_local_table(
    table_name=table_name,
    namespace="demo",
    db_name="demo_catalog",
)
store = AdapterStore(table, default_namespace="snapshot_namespace")
print(f"Created table: {table_name}")

In [ ]:
def create_record_table(records: list[dict], namespace: str = "snapshot_namespace") -> pa.Table:
    """Create a PyArrow table from a list of records.
    
    Note: snapshot_sync does NOT require timestamps, but will auto-generate them if missing.
    You can optionally provide last_modified values in records.
    """
    for record in records:
        record["namespace"] = namespace
    
    # Basic schema without timestamps (snapshot_sync doesn't require them)
    schema = pa.schema([
        pa.field("namespace", pa.string(), nullable=False),
        pa.field("id", pa.string(), nullable=False),
        pa.field("content", pa.string(), nullable=True),
    ])
    
    return pa.Table.from_pylist(records, schema=schema)

def display_table(title: str):
    """Display current table contents."""
    print(f"\n{title}")
    print("=" * 100)
    current = table.scan().to_arrow().sort_by("id")
    if current.num_rows == 0:
        print("(empty table)")
    else:
        df = current.to_pandas()
        # Format for better readability
        print(df.to_string(index=False))
    return current

def display_changeset_summary():
    """Display a summary of changesets in the table."""
    print("\n📊 Changeset Summary")
    print("=" * 100)
    current = table.scan().to_arrow()
    if current.num_rows == 0:
        print("(empty table)")
        return
    
    df = current.to_pandas()
    # Group by changeset
    changeset_counts = df.groupby('changeset', dropna=False).size()
    print(f"\nTotal records: {len(df)}")
    print(f"Changesets: {changeset_counts.to_dict()}")
    
    # Show deleted records count
    deleted_count = df['content'].isna().sum()
    if deleted_count > 0:
        print(f"🗑️  Deleted records (content=None): {deleted_count}")

## Scenario 1: Initial Load into Empty Table

Starting with an empty table, load initial records.

In [ ]:
initial_snapshot = create_record_table([
    {"id": "rec001", "content": "First record"},
    {"id": "rec002", "content": "Second record"},
    {"id": "rec003", "content": "Third record"},
])

changeset_id = store.snapshot_sync(initial_snapshot)
print(f"✅ Initial load complete. Changeset ID: {changeset_id}")

current = display_table("Initial State")
display_changeset_summary()

# Verify all records have the same changeset
assert current.column("changeset").to_pylist().count(changeset_id) == 3
print("\n✓ All records share the same changeset_id")

## Scenario 2: Idempotent Sync (No Changes)

Apply the same snapshot again. Should return `None` and make no changes.

In [ ]:
same_snapshot = create_record_table([
    {"id": "rec001", "content": "First record"},
    {"id": "rec002", "content": "Second record"},
    {"id": "rec003", "content": "Third record"},
])

result = store.snapshot_sync(same_snapshot)

if result is None:
    print("✅ No changes detected (idempotent)")
else:
    print(f"❌ Unexpected: changeset created: {result}")

display_table("After Idempotent Sync (should be unchanged)")
display_changeset_summary()

# Verify no new records created
assert current.num_rows == 3
print("\n✓ Record count unchanged")

## Scenario 3: Content Updates

Update some existing records with different content.

In [ ]:
updated_snapshot = create_record_table([
    {"id": "rec001", "content": "UPDATED first record"},
    {"id": "rec002", "content": "Second record"},  # unchanged
    {"id": "rec003", "content": "UPDATED third record"},
])

changeset_id_2 = store.snapshot_sync(updated_snapshot)
print(f"✅ Updates applied. Changeset ID: {changeset_id_2}")

current = display_table("After Content Updates")
display_changeset_summary()

# Verify only changed records have new changeset
records = current.to_pylist()
rec001 = [r for r in records if r["id"] == "rec001"][0]
rec002 = [r for r in records if r["id"] == "rec002"][0]
rec003 = [r for r in records if r["id"] == "rec003"][0]

assert rec001["changeset"] == changeset_id_2, "rec001 should have new changeset"
assert rec002["changeset"] == changeset_id, "rec002 should retain old changeset (unchanged)"
assert rec003["changeset"] == changeset_id_2, "rec003 should have new changeset"

print("\n✓ Only changed records received new changeset_id")

## Scenario 4: Insertions

Add new records to the snapshot.

In [ ]:
snapshot_with_new = create_record_table([
    {"id": "rec001", "content": "UPDATED first record"},
    {"id": "rec002", "content": "Second record"},
    {"id": "rec003", "content": "UPDATED third record"},
    {"id": "rec004", "content": "NEW fourth record"},
    {"id": "rec005", "content": "NEW fifth record"},
])

changeset_id_3 = store.snapshot_sync(snapshot_with_new)
print(f"✅ Insertions applied. Changeset ID: {changeset_id_3}")

current = display_table("After Insertions")
display_changeset_summary()

# Verify new records exist and have the new changeset
records = current.to_pylist()
rec004 = [r for r in records if r["id"] == "rec004"][0]
rec005 = [r for r in records if r["id"] == "rec005"][0]

assert rec004["changeset"] == changeset_id_3
assert rec005["changeset"] == changeset_id_3
assert current.num_rows == 5

print("\n✓ New records inserted with correct changeset_id")

## Scenario 5: Soft Deletes

Remove records from the snapshot. They should be marked as deleted (`content=None`) rather than physically removed.

In [ ]:
snapshot_with_deletions = create_record_table([
    {"id": "rec001", "content": "UPDATED first record"},
    {"id": "rec002", "content": "Second record"},
    # rec003, rec004, rec005 are missing - should be deleted
])

changeset_id_4 = store.snapshot_sync(snapshot_with_deletions)
print(f"✅ Deletions applied. Changeset ID: {changeset_id_4}")

current = display_table("After Deletions")
display_changeset_summary()

# Verify deleted records have content=None
records = current.to_pylist()
rec003 = [r for r in records if r["id"] == "rec003"][0]
rec004 = [r for r in records if r["id"] == "rec004"][0]
rec005 = [r for r in records if r["id"] == "rec005"][0]

assert rec003["content"] is None, "rec003 should be soft-deleted"
assert rec004["content"] is None, "rec004 should be soft-deleted"
assert rec005["content"] is None, "rec005 should be soft-deleted"
assert rec003["changeset"] == changeset_id_4
assert rec004["changeset"] == changeset_id_4
assert rec005["changeset"] == changeset_id_4

print("\n✓ Deleted records marked with content=None")
print("✓ All deleted records have the same changeset_id")

# Show deleted records separately
print("\n🗑️  Deleted Records:")
deleted_records = [r for r in records if r["content"] is None]
for rec in deleted_records:
    print(f"  - {rec['id']} (changeset: {rec['changeset'][:8]}...)")

## Scenario 6: Undelete / Restore

Restore a previously deleted record. This should update the existing deleted record (not create a duplicate).

## Scenario 7: Mixed Operations (Insert + Update + Delete)

Apply a snapshot with all three operations simultaneously.

In [ ]:
snapshot_with_restore = create_record_table([
    {"id": "rec001", "content": "UPDATED first record"},
    {"id": "rec002", "content": "Second record"},
    {"id": "rec003"},  # Deleting this one
])

changeset_id_5 = store.snapshot_sync(snapshot_with_restore)

current = display_table("After Update")
display_changeset_summary()
records = current.to_pylist()
rec003 = records[2]  # Assuming sorted by id

assert "content" not in rec003 or rec003["content"] is None
assert current.num_rows == 5, "Total records should still be 5"

print("\n✓ Update successful")
print(f"✓ Total records: {current.num_rows}")

## Summary

This notebook demonstrated the complete behavior of `snapshot_sync`:

### ✅ What We Showed

1. **Initial Load**: Populating an empty table
2. **Idempotency**: Repeated syncs with identical data produce no changes
3. **Updates**: Content changes are detected and applied
4. **Insertions**: New records are added
5. **Soft Deletes**: Missing records marked with `content=None` (not physically deleted)
6. **Undelete/Restore**: Deleted records can be restored without creating duplicates
7. **Mixed Operations**: Insert, update, and delete in a single sync
8. **Namespace Isolation**: Only the specified namespace is affected
9. **Query Methods**: Different ways to retrieve records (by changeset, active only, including deleted)

### Key Differences from `incremental_update`

| Feature | `snapshot_sync` | `incremental_update` |
|---------|-----------------|----------------------|
| **Deletions** | ✅ Deletes missing records | ❌ Never deletes |
| **Timestamps** | ⚠️ Writes them but doesn't check | ✅ Required & enforced (newer only) |
| **Use Case** | Full synchronization | Incremental harvesting |
| **Data Protection** | None (overwrites freely) | Protects against timestamp regression |

### When to Use Each Method

- **Use `snapshot_sync`**: When you have a complete snapshot and want the table to match exactly (e.g., nightly full dumps, complete exports)
- **Use `incremental_update`**: When harvesting changes incrementally where records may arrive out of order (e.g., OAI-PMH, event streams)

## Cleanup

In [ ]:
# Drop the demo table
table.catalog.drop_table(f"demo.{table_name}")
print(f"✅ Dropped table: {table_name}")